In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=720):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "017960"
    company_name = "한국카본"

    # 데이터 수집
    df = get_day_price(code, max_pages=720)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜      종가  전일비      시가      고가      저가    거래량
2025.04.25 19750.0 -350 20150.0 20550.0 19680.0 697581
2025.04.24 20100.0 1090 18990.0 20300.0 18650.0 932509
2025.04.23 19010.0   80 18950.0 19060.0 18610.0 246250
2025.04.22 18930.0  260 18430.0 19060.0 18430.0 399278
2025.04.21 18670.0 -210 18690.0 18690.0 17970.0 609070
2025.04.18 18880.0 -120 18920.0 18920.0 18370.0 442825
2025.04.17 19000.0  190 18820.0 19170.0 18740.0 475858
2025.04.16 18810.0 -710 19210.0 19280.0 18450.0 898222
2025.04.15 19520.0  300 19400.0 19540.0 18890.0 723531
2025.04.14 19220.0  170 19430.0 19510.0 18800.0 870332
2025.04.11 19050.0  240 19000.0 19490.0 18760.0 1262907
2025.04.10 18810.0 1370 18200.0 18840.0 17600.0 1113480
2025.04.09 17440.0  130 18200.0 18270.0 17110.0 1406023
2025.04.08 17310.0 1250 16740.0 17460.0 16570.0 1518605
2025.04.07 16060.0 -770 16500.0 16750.0 15790.0 1432384
2025.04.04 16830.0  280 16490.0 17400.0 16260.0 1626654
2025.04.03 16550.0 1300 15020.0 16550.0 15010.0 1839062
202